# Collection of tweets on birds from Twitter.com using REST API - Github

## Alexindata 04/12/2017

## Tools: Jupyter Notebook with Python 3.5, MongoDB, PostgreSQL RDBMS


In [1]:
import time
import datetime
import psycopg2
import pymongo
import json
import re
import tweepy

from IPython.display import display
from bson.objectid import ObjectId
from pymongo import MongoClient
from tweepy import OAuthHandler

In [2]:
# use per-app Twitter OAuth

consumer_key = '######'
consumer_secret = '######'

auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


## Select birds from ebird psql table to search on Twitter.com

In [5]:
# this selects all birds in ebird postgresql db

user = '######'
pw = "######"
conn = psycopg2.connect(dbname = "birds", host = "localhost", port = 5432, user = user, password = pw)
cur = conn.cursor()

try:
    cur.execute("SELECT name FROM ebird;")
except:
    conn.rollback()
else:
    bird_list = cur.fetchall()
    print('Bird list selected.')

all_birds = [b[0] for b in bird_list]  # de-tupling
display(all_birds[:3])  # print the first 3 birds of the list

cur.close()
conn.close()


Bird list selected.


['American Black Duck',
 'American Three toed Woodpecker',
 'American Tree Sparrow']

In [23]:
# connect to MongoDB database named 'twitter'

client = MongoClient()
db = client.twitter  # mongo in mongohub is the name of the connection


## Generate variants of bird names used in search

### Example: Yellow rumped Warbler

* Yellow rumped Warbler
* Yellow rumped #Warbler
* #yellowrumpedwarbler

_'All these words' option is used in Twitter REST API.search (not 'Exact phrase' option)_

In [7]:
# a function that generate variants of bird names with hashtags

def hashtag(bird):
    bird_tag = set()
    bird_tag.add(bird)
    bird_tag.add('#' + ''.join(bird.split(' ')))
    
    s = bird.split(' ')
    s[-1] = '#' + s[-1]
    bird_tag.add(' '.join(s))
    return bird_tag

In [28]:
# download tweets from Twitter.com
# load tweets into MongoDB 'twitter', 'tweets' collection
# load tweet id, MongoDB _id, search term, search timestamp info into psql db 'birds', 'tweets' table

last_id = -1
max_id = None
total_tweets_saved = 0

user = '######'
pw = "######"
conn = psycopg2.connect(dbname = "birds", host = "localhost", port = 5432, user = user, password = pw)
cur = conn.cursor()

try:
    cur.execute("SELECT MAX(CAST(id_str AS bigint)) FROM tweets;")
except:
    conn.rollback()
else:
    max_id = cur.fetchone()[0]
    print('Starting timestamp: ', datetime.datetime.now(), 'Max_id: ', max_id)

excluded_words = ['MLB', 'Baseball', 'Ticket', '500px', 'Art', 'Artwork', 'Drawing', 'Painting', 'Sale', 'Offer']
excluded_birds = ['Canada Goose']

bird_list = all_birds

for bird in bird_list:
    if bird in excluded_birds: continue

    tweets_saved = 0

    # tweet id are BIGINT, which is out of range for postgresql int type
    # must use (bird, ) and not (bird) in cur.execute() phrase    
    try:
        cur.execute("SELECT search_term, MAX(CAST(id_str AS bigint)) FROM tweets WHERE search_term = %s GROUP BY search_term;", (bird,))
    except:
        conn.rollback()
    else:
        try:
            last_id = cur.fetchone()[1]
        except:
            last_id = -1
    print(bird, last_id)
    
    bird_tag = hashtag(bird)
    for tag in bird_tag:

        # use twitter search api to search, filter, and load raw tweets into mongo.twitter mongodb
        for status in tweepy.Cursor(api.search, q=tag, result_type='mixed', since_id = max_id, include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True).items(1000):

            _id = None
            media = None
            exclude_flag = 0
            
            tweet = status._json
            id_str = tweet['id_str']
            text = tweet['text']

            # is the tweet a retweet?
            if text.startswith('RT '):
                exclude_flag = 1
            else:
                try:
                    if tweet['retweeted_status']:
                        exclude_flag = 1
                except:
                    exclude_flag = 0
            
            # does the tweet contain excluded words? 
            for word in excluded_words:
                if re.search(word.lower(), text.lower()):
                    exclude_flag = 1
                    
            if re.search('^#photography #.+ by', text.lower()):
                exclude_flag = 1
                
            if exclude_flag == 1:
                #print('This tweet is excluded: ', text)
                continue

            # only store tweets that are not retweets, in english, has photo-video media, and not including certain words

            retweet_count = tweet['retweet_count']
            lang = tweet['lang']

            if tweet['entities'].get('media', None) is not None:
                media = tweet['entities']['media'][0]['type']

            if db.tweets.find_one({'id_str': id_str}):
                continue

            if (lang == 'en') and (media in ['photo', 'video']):                               
                ### insert tweepy status object as json format in mongo db ###
                try:
                    _id = db.tweets.insert_one( tweet ).inserted_id  # _id is an ObjectId object, use str() to convert to str
                    if _id:
                        try:
                            cur.execute("INSERT INTO tweets (id_str, mongo_id, retweet_count, search_term, media) VALUES (%s, %s, %s, %s, %s);", (id_str, str(_id), retweet_count, bird, media))
                        except psycopg2.Error as e:
                            conn.rollback()
                            print('Inserted into MongoDB, but rolled back in Psql db, _id, with error ', _id, e)
                        else:
                            conn.commit()
                            tweets_saved += 1

                except:
                    print('Error with MongoDB, db.collection.insert_one')

    print('{} tweets saved in mongodb and postgresql databases for {}.'.format(tweets_saved, bird))
    total_tweets_saved += tweets_saved
                                     
print('{} tweets saved in mongodb and postgresql databases in total.'.format(total_tweets_saved))
print('Ending timestamp: ', datetime.datetime.now())

cur.close()
conn.close()


## Display some tweets text in Mongodb

In [8]:
n = 0
for obj in db.tweets.find():
    n += 1
    if n > 3000 and n < 3010:
        print(obj['text'], obj['id'])
    else:
        continue

Mayuko sighted a Mourning Dove on NatureShare https://t.co/BBtlxGSgDv https://t.co/MOGE4EhGMF 847571894924697601
"The Cooing of the Dove and the Cawing of the Crow: The Poetics of Mourning in Two Elegies" https://t.co/gfAtgElwnP https://t.co/Lze89HKSDW 847461530316623881
@wildtimes we found an egg in our yard after the storm. Think it's a mourning dove. Suggestions? https://t.co/mgdq8PPhjp 847447758092947459
Beautiful wing feather detail of a Mourning Dove 03/30 09:2 #cornellfeeders https://t.co/QLcaHkQCvD 847446684657270785
Rare sighting of the majestic mourning dove #oxybirds https://t.co/hCfwvwecYT 847310942819950594
My friend Jack the Mourning Dove. https://t.co/QUfFZx8DUF 847267983453847552
Mourning Dove blocks your path. https://t.co/nZ8Ld7jHAY 847230457313804290
Mourning Dove couple in my front yard 💕 https://t.co/egGBm3Y4CF 847208218933809152
Mourning Warbler https://t.co/MYgZqwioOl 850061792021811200


In [10]:
# total number of tweets in Mongodb as-of-now
print(db.tweets.find({}).count())

6442


## Finding tweets in mongodb

In [16]:
# by searching for words in tweet text
for t in db.tweets.find( {'$and' : [{'text' : {'$regex' : '.*Townsend\'s.*', '$options' : 'i'}}, 
                                    {'text' : {'$regex' : '.*Warbler.*', '$options': 'i'}}]}):
    print(t['text'] )


Learn about Townsend's Warbler behavior, habitat, and more in our free online bird guide: https://t.co/bwucORywhK https://t.co/HaPQa4hlbm
Townsend's Warbler (Pismo Beach, CA) via /r/birdpics https://t.co/cxUDLYxWR9 https://t.co/ysjNkVhfUn


In [18]:
# show 1 random tweet, displaying the whole object
db.tweets.find_one()

{'_id': ObjectId('58e8dfc340383b024ee905f5'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Apr 06 12:03:23 +0000 2017',
 'entities': {'hashtags': [{'indices': [21, 30], 'text': 'Brooklyn'},
   {'indices': [62, 66], 'text': 'nyc'},
   {'indices': [67, 75], 'text': 'birding'}],
  'media': [{'display_url': 'pic.twitter.com/cbwW73SkKl',
    'expanded_url': 'https://twitter.com/heatherwolf/status/849955692710637568/photo/1',
    'id': 849955600926744577,
    'id_str': '849955600926744577',
    'indices': [76, 99],
    'media_url': 'http://pbs.twimg.com/media/C8umHaQXoAEFaRA.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/C8umHaQXoAEFaRA.jpg',
    'sizes': {'large': {'h': 725, 'resize': 'fit', 'w': 1000},
     'medium': {'h': 725, 'resize': 'fit', 'w': 1000},
     'small': {'h': 493, 'resize': 'fit', 'w': 680},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'type': 'photo',
    'url': 'https://t.co/cbwW73SkKl'}],
  'symbols': [],
  'urls': [],
  'use

In [19]:
# find the tweet by id_str, displaying the whole object
db.tweets.find_one({'id_str': '850061792021811200'})

{'_id': ObjectId('58e906c340383b024ee911fe'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Apr 06 19:04:59 +0000 2017',
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/MYgZqwioOl',
    'expanded_url': 'https://twitter.com/birdcolourbot/status/850061792021811200/photo/1',
    'id': 850061789018689538,
    'id_str': '850061789018689538',
    'indices': [17, 40],
    'media_url': 'http://pbs.twimg.com/media/C8wGsXuXkAIIYjF.png',
    'media_url_https': 'https://pbs.twimg.com/media/C8wGsXuXkAIIYjF.png',
    'sizes': {'large': {'h': 100, 'resize': 'fit', 'w': 300},
     'medium': {'h': 100, 'resize': 'fit', 'w': 300},
     'small': {'h': 100, 'resize': 'fit', 'w': 300},
     'thumb': {'h': 100, 'resize': 'crop', 'w': 100}},
    'type': 'photo',
    'url': 'https://t.co/MYgZqwioOl'}],
  'symbols': [],
  'urls': [],
  'user_mentions': []},
 'extended_entities': {'media': [{'display_url': 'pic.twitter.com/MYgZqwioOl',
    'expanded_url': 'https://t

## Quality check on downloaded tweets

### Problematic search terms:

* Baltimore Oriole: 119/7112 tweets collected, confused with the MLB team
* Redhead: 344/7112 tweets collected, women that are redheads,
* Brant: place name, people name
* Osprey: brands
* Canada Goose: commercial brand
* Merlin: a priest

### Solutions:
* remove all rows collected by these search terms from both mongodb and postgresql db
* change in both ebird and wiki tables: 
    * Redhead to Redhead Duck, 
    * Brant to Brant Goose, 
    * Osprey to Osprey Eagle 
    * Merlin to Merlin Falcon
* exclude tweets if containing some keywords, e.g., artwork, sale, painting, MLB etc
* exclude some bird names from tweet collection, e.g. Canada Goose


# Interactions between MongoDB and Postgresql db

## From MongoDB to Postgresql db

### Delete records in MongoDB by a field attribute, use the \_id to delete records in Postgresql db

* Delete tweets containing 'Offer', or 'offer' in tweet text

In [22]:
user = '######'
pw = '######'
conn = psycopg2.connect(dbname = "birds", host = "localhost", port = 5432, user = user, password = pw)
cur = conn.cursor()

# find records containing 'offer' or 'Offer' in tweet text
# get the _id back

for t in db.tweets.find( {"text": {"$regex": ".*offer.*|.*Offer.*"}} ):
    # t['_id] is of type bson.objectid.ObjectId, need to be converted to str for use in postgresql query
    try:
        cur.execute("DELETE FROM tweets WHERE mongo_id = %s;", (str(t['_id']), ))
    except:
        conn.rollback()
        print("not working...")
    else:
        conn.commit()
        print('Psgl successfully committed.')
    result = db.tweets.delete_one( {'_id':t['_id']} )
    print(result.deleted_count)

cur.close()
conn.close()

## From Postgresql db to MongoDB

* Find the ids of the tweets retrieved from twitter, under search term : "Anna's Hummingbird" in Postgresql db
* Retrieve the tweet text from Mongodb


In [20]:
user = '######'
pw = "######"
conn = psycopg2.connect(dbname = "birds", host = "localhost", port = 5432, user = user, password = pw)
cur = conn.cursor()

try:
    cur.execute("SELECT mongo_id FROM tweets WHERE LOWER(search_term) = LOWER('Anna''s Hummingbird');")
except:
    conn.rollback()
else:
    result = cur.fetchall()
    print('Psgl successfully committed.')

# search mongodb using '_id'
# must use ObjectId(_id) from 
# result = db.warbler.delete_one({"_id":ObjectId("58e6284140383b02a9515c64")})
# result

for _id in [i[0] for i in result]:
    t = db.tweets.find_one( {"_id" : ObjectId(_id)} )
    print(t['text'])

Psgl successfully committed.
Anna's Hummingbird #desert #southwest #Arizona #hummingbird #photography https://t.co/jyzHmPZimA
Female Anna's Hummingbird h40 #photography by Mark Myhaver #myhaverphotography #hummingbird https://t.co/3xS4RGqcNm https://t.co/Xq3iWcBs6s
Anna's #hummingbird at my patio feeder. #birdphotography #photography https://t.co/gqA27hbRCU
Anna’s Hummingbird feeding on a flower blossom. #NationalPollinatorMonth https://t.co/vmnkNtCik3
Learn about Anna's Hummingbird behavior, habitat, and more in our free online bird guide: https://t.co/LyrUaPNzF7 https://t.co/Xo24jODxmk
Did someone say tweet? An Anna's hummingbird outside my house. https://t.co/huKl50vhCT
white Anna's Hummingbird with leucatism (not albinism) at UCSC Arboretum https://t.co/PGYT5Qrnth
Anna's Hummingbird
Grenvillea Rosemary
@TurtleBayPark https://t.co/NCb5IEt6nt
Kent Washington USA https://t.co/Bt0hkgxucG
We had some nice yard birds this afternoon. Red-tailed hawk, Anna's hummingbird and Costa's humming

## Generate the web link to the tweets of interest on Twitter.com

In [27]:
base_url = 'https://twitter.com/anyuser/status'

for _id in [i[0] for i in result]:
    t = db.tweets.find_one( {"_id" : ObjectId(_id)} )
    print('{}/{}'.format(base_url, t['id_str']))

https://twitter.com/anyuser/status/849755499885211648
https://twitter.com/anyuser/status/849622872402391041
https://twitter.com/anyuser/status/847704474961330177
https://twitter.com/anyuser/status/848535741655375873
https://twitter.com/anyuser/status/847065873592565761
https://twitter.com/anyuser/status/850491924930678785
https://twitter.com/anyuser/status/850883654725230596
https://twitter.com/anyuser/status/850818231946231808
https://twitter.com/anyuser/status/850738275505635328
https://twitter.com/anyuser/status/851197774297157632
